# **Tabular playground Series _ Nov 2021**

## **Baseline :  STEPS**

### ***First stage apporach of Problem --> Notebook_Boosting techniques***

1. Import CV-KFOLD-Split5 Data
1. Read the data
2. Identify the null data
3. Feature Selection
4. Preprocessing the data
5. Split the data(Train_Test_Split)
6. Build the Model(I'm try XGBRegressor)
7. XGBRegressor
8. Catboost classifier
9. LGBM
10. ROC and Save csv file

## **Import Necessary libraries** 

In [ ]:
#Import necessay libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
import seaborn as sns

#Preprocessing
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,roc_auc_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import xgboost as xgb


## ***Bulid the Model***

In [ ]:
#import the data and shape
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")
sample=pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
print(train.shape,test.shape,sample.shape)
print(train.isnull().sum())
train.describe()

## **XGBoostRegressor_KFold_5 data** 
**I'm trying optuna and tune the hyperparameters the value of output is params_b**


In [ ]:
train = pd.read_csv("../input/tps-nov-21-crossvalidationdata/trainfold_5.csv")
#store the final_prediction data and score
final_predictions = []
score= []

#features(categorical and numerical datas separate)
useful_features = [c for c in train.columns if c not in ("id","target","kfold")]
numerical_cols = [col for col in useful_features if 'f' in col]

test = test[useful_features]

for fold in range(5):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    scaler = preprocessing.StandardScaler()
    xtrain[numerical_cols] = scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols] = scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    
    params_b = {
        'eta':0.0006409492043348976,
        'gamma': 0.44055233553496753,
        'reg_lambda':0.000669870160395597,
        'reg_alpha':9570.52928031902,
        'subsmaple':0.6048519345751173,
        'learning_rate':0.01,
        'n_estimators':10000,
        
        }
    model= XGBRegressor(**params_b,
                       tree_method='gpu_hist',
                       predictor='gpu_predictor',
                       gpu_id=0)
    model.fit(xtrain,ytrain,early_stopping_rounds=300,eval_set=[(xvalid,yvalid)],verbose=2000)
    preds_valid = model.predict(xvalid)
    
    #Training model apply the test data and predict the output
    test_pre = model.predict(xtest)
    final_predictions.append(test_pre)
    
    roc=roc_auc_score(yvalid,preds_valid)
    score.append(roc)
    #way of output is display
    print(f"fold:{fold},roc:{roc}")

#mean of repeation of fold data and identify the  mean and standard deviation 
print(np.mean(score),np.std(score))


In [ ]:
#prediction of data
preds = np.mean(np.column_stack(final_predictions),axis=1)
print(preds)
sample.target = preds
sample.to_csv("submission_XG.csv",index=False)
print("success")

In [ ]:
#Optuna
import optuna
from optuna.samplers import TPESampler
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe


def objective(trial):
    params = {'max_depth':hp.quniform("max_depth",3,10,2),
              'eta': trial.suggest_loguniform('eta',1e-5,0.2),
              'gamma':trial.suggest_loguniform('gamma',1e-8,1.0),
              'reg_lambda': trial.suggest_loguniform('reg_lambda',1e-4,1e4),
              'reg_alpha': trial.suggest_loguniform('reg_alpha',1e-4,1e4),
              'colsample_bytree': hp.uniform('colsample_bytree',0.1,1.1),
              'min_child_weight':hp.quniform('min_child_weight',100,1000,1),
              'subsample': trial.suggest_float('subsample',1e-8,1),
              'learning_rate': trial.suggest_loguniform("learning_rate",0.001,0.3),
              'max_depth': trial.suggest_int("max_depth", 1, 10),
              'n_estimators':10000,
              'seed':0,
              'tree_method':'gpu_hist',
              'gpu_id': 0,
              'predictor': 'gpu_predictor'
            }
    
    model = xgb.XGBRegressor(**params)

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)], 
        verbose=False
    )

    yhat = model.predict(xvalid)
    return roc_auc_score(yvalid, yhat)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_params)

In [ ]:
params= study.best_params

In [ ]:
model= XGBRegressor(**params,
                    tree_method='gpu_hist',
                    predictor='gpu_predictor',
                    gpu_id=0)
model.fit(xtrain,ytrain,early_stopping_rounds=300,eval_set=[(xvalid,yvalid)],verbose=2000)
preds_valid = model.predict(xvalid)
    
#Training model apply the test data and predict the output
test_pre = model.predict(xtest)

sample.target = test_pre
sample.to_csv("submission_XG1.csv",index=False)
print("success")

## **CatBoostClassifier** 

In [ ]:
from catboost import CatBoostClassifier
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
#store the final_prediction data and score
final_prediction1 = []
score= []

features = [f for f in train.columns if f not in("id","target")]
X = train[features]
y = train.target
test = test[features]

ss = StandardScaler()
X = ss.fit_transform(X)
xtest = ss.transform(test)

xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=42)
print(xtrain.shape,ytrain.shape,xvalid.shape,yvalid.shape)

c_para = {
    'iterations':25000,
    'loss_function':'Logloss',
    'use_best_model':True,
    'early_stopping_rounds':1000,
    'learning_rate':0.0513589,
    'border_count':32,
    'verbose':1000,
    'random_state':228,
    'subsample': 0.95312,
    "max_depth": 2,
    "min_data_in_leaf":77,
    'rsm':0.5,
    'l2_leaf_reg': 0.02247766515106271
}
cat_boost = CatBoostClassifier(**c_para)
cat_boost.fit(xtrain,ytrain,eval_set=[(xvalid,yvalid)])

pred = cat_boost.predict(xvalid)
test_pred = cat_boost.predict(xtest)
final_prediction1.append(test_pred)

roc=roc_auc_score(yvalid,pred)
score.append(roc)
#way of output is display
print(f"fold:{fold},roc:{roc}")

#prediction of data
preds = np.column_stack(final_prediction1)
print(preds)
sample.target = preds
sample.to_csv("submission_CAT.csv",index=False)
print("success")

## **LGBM** 

In [ ]:
prediction1=[]
score = []
#lgb parameters
params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    'subsample': 0.95312,
    'learning_rate': 0.0051635,
    "max_depth": 4,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.7705303688019942,
    "min_child_samples": 290,
    "reg_alpha": 14.68267919457715,
    "reg_lambda": 66.156,
    "max_bin": 772,
    "min_data_per_group": 177,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 17,
    "verbosity": -1,
    'random_state':288,
    'n_estimators':8000,
    'colsample_bytree':0.1107,
    'njobs':4
}
    
lgb_train = lgb.Dataset(xtrain, ytrain)
lgb_val = lgb.Dataset(xvalid, yvalid)
    
model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=1000)
    
preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
test_predict = model.predict(xtest,num_iteration=model.best_iteration)
prediction1.append(test_predict)
#ROC
roc=roc_auc_score(yvalid,preds_valid)
#Score 
score.append(roc)
print(np.mean(score),np.std(score))

#prediction of data
preds = np.column_stack(prediction1)
print(preds)
sample.target = preds
sample.to_csv("submission_LGB.csv",index=False)
print("success")

## **Thankyou**  😊 😊

**I am trying different Boosting techniques and various hyperparameters** 😊 😊 😊